<a href="https://colab.research.google.com/github/roccaab/WaveletGAN/blob/main/decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def wavelet_decomposition(signal, wavelet='db4', level=6):
    """
    Esegue la decomposizione wavelet su un batch di segnali.

    Input:
      - signal: torch.Tensor di forma (batch, 2000)
      - wavelet: nome della wavelet (default 'db4')
      - level: numero di livelli (default 6)

    Output:
      - coeffs_torch: torch.Tensor di forma (batch, 6, 2000)
        (per ogni segnale, 6 componenti di dettaglio ricampionate alla lunghezza originale)
    """
    batch_size = signal.shape[0]
    coeffs_batch = []
    for i in range(batch_size):
        # Calcola la decomposizione wavelet; coeffs[0] è l'approssimazione, usiamo i dettagli
        coeffs = pywt.wavedec(signal[i].cpu().numpy(), wavelet, level=level)
        # Utilizziamo solo i coefficienti di dettaglio (6 livelli)
        coeffs_resampled = []
        for c in coeffs[1:]:
            # upcoef converte i coefficienti in un segnale della lunghezza desiderata (2000)
            rec = pywt.upcoef('d', c, wavelet, level=level, take=len(signal[i]))
            coeffs_resampled.append(torch.tensor(rec, dtype=torch.float))
        # Stack: forma (6, 2000)
        coeffs_batch.append(torch.stack(coeffs_resampled))
    coeffs_torch = torch.stack(coeffs_batch).to(signal.device)
    return coeffs_torch